## Data Mining Project 

Ariel Mannes - 302229190 , Eliran Tal - 200680676 , 

Q1: for the data exploration we used only the genders with 100% certenty. We found the common words for: Male ,Female and Brand.

In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.utils import shuffle
from nltk.probability import FreqDist
import string
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from zmq.backend.cython import message
import os
os.environ["MKL_THREADING_LAYER"] = "GNU"
os.environ["KERAS_BACKEND"] = "theano"
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer as keras_token

Using Theano backend.


In [2]:
# fix random seed for reproducibility
np.random.seed(7)

ps = PorterStemmer()
stop_words = stopwords.words('english') + list(string.punctuation)

## import dataset
df = pd.read_csv('gender-classifier-DFE-791531.csv', encoding='latin1')
df = shuffle(shuffle(shuffle(df)))
df


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
8284,815727607,False,finalized,3,10/26/15 23:00,brand,0.6691,yes,1.0,4/22/14 5:16,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,105610,10/26/15 12:39,6.587300e+17,NaN,NaN
88,815719314,False,finalized,3,10/27/15 1:38,brand,0.6629,yes,1.0,3/30/09 23:08,...,https://pbs.twimg.com/profile_images/183683348...,0,BDDCAD,Goodword @ The Honey Hive Gallery https://t.co...,NaN,7110,10/26/15 12:40,6.587300e+17,"Berkeley, ca",Central Time (US & Canada)
757,815719986,False,finalized,3,10/27/15 0:36,female,1.0000,yes,1.0,4/8/13 3:17,...,https://pbs.twimg.com/profile_images/635650111...,0,FFFFFF,the thunder was so hectic last night - shook m...,NaN,527,10/26/15 12:40,6.587300e+17,"Byron Bay, NSW",Brisbane
4583,815723885,False,finalized,3,10/27/15 0:09,brand,1.0000,yes,1.0,8/20/13 9:48,...,https://pbs.twimg.com/profile_images/378800000...,0,0,Click the link to see what Rev. John Massey Jr...,NaN,657,10/26/15 12:40,6.587300e+17,"Northfield, MA",NaN
8852,815728187,False,finalized,3,10/27/15 1:43,male,1.0000,yes,1.0,5/30/15 10:29,...,https://pbs.twimg.com/profile_images/647694406...,0,C0DEED,The NBA is back tomorrow Ï__Ù_,NaN,3065,10/26/15 12:39,6.587300e+17,"Crockett, TX Ïöü San Diego, CA",NaN
10635,815730002,False,finalized,3,10/27/15 1:29,male,1.0000,yes,1.0,10/31/14 21:05,...,https://pbs.twimg.com/profile_images/639173126...,0,0,whats with everyone making cake that look like...,NaN,298,10/26/15 13:20,6.587400e+17,"Noblesville,IN",NaN
18874,815738442,False,finalized,3,10/27/15 0:34,brand,0.6636,yes,1.0,3/28/15 3:55,...,https://pbs.twimg.com/profile_images/642529279...,0,C0DEED,EVERYDAYILOVEYOU Forevermore\n\nand what makes...,NaN,108475,10/26/15 13:19,6.587400e+17,NaN,Singapore
9352,815728697,False,finalized,3,10/26/15 21:36,male,1.0000,yes,1.0,6/10/13 18:51,...,https://pbs.twimg.com/profile_images/647139155...,0,C0DEED,nobody knows my struggle they only see the tro...,NaN,33674,10/26/15 12:39,6.587300e+17,NaN,Pacific Time (US & Canada)
12644,815732066,False,finalized,3,10/27/15 1:12,female,0.6708,yes,1.0,9/13/12 19:15,...,https://pbs.twimg.com/profile_images/372087887...,0,FFF8AD,THERE IS ZERO DIFFERENCE between GOPe and Hill...,NaN,106653,10/26/15 13:20,6.587400e+17,SoCal,Pacific Time (US & Canada)
7345,815726663,False,finalized,3,10/26/15 23:54,brand,0.6813,yes,1.0,7/2/14 6:31,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,68830,10/26/15 12:40,6.587300e+17,NaN,NaN


In [3]:
## Question 1


## text pre-processing
dist = df.groupby('gender').size().to_frame()


# compute gender disterbution before text processing
def compute_percentage(x):
    pct = float(x / df['gender'].size) * 100
    return round(pct, 2)


dist['percentage'] = dist.apply(compute_percentage, axis=1)
dist

# remove points we're not confident about
filtered = df.values.tolist()
filtered = list(filter(lambda x: x[6] == 1.0 and x[5] != "unknown", filtered))

# split our data to three smaller lists by gender
males = list(filter(lambda x: x[5] == "male", filtered))
females = list(filter(lambda x: x[5] == "female", filtered))
brands = list(filter(lambda x: x[5] == "brand", filtered))


def try_str(t):
    try:
        return str(t)
    except:
        return ""

        # remove characters like "'" or "."


def wordify(t):
    t = t.replace("'", "")
    t = t.replace(",", " ")
    t = t.replace(".", " ")
    t = t.replace("!", " ")
    t = t.replace("?", " ")
    t = t.replace("&", " ")
    t = t.replace("|", " ")
    t = t.replace("/", "")
    t = t.replace(";", " ")
    t = t.replace(":", " ")
    t = t.replace("\\", "")
    t = t.replace("\\n", " ")
    return t


# combine descriptions and text tweets into a single list of 'words'
def get_list_of_words(lst):
    word_list = " ".join(list(map(lambda x: try_str(x[10]), lst)))
    word_list = " ".join(list(map(lambda x: try_str(x[19]), lst)))
    word_list = word_list.lower().split(" ")
    # word_list = wordify(word_list).split(" ")
    word_list = list(w for w in word_list if len(w) > 1 and w not in stop_words)
    return word_list


def word_distrebution(text, messeg, f):
    ret_list = []
    list_words = " ".join(list(map(f, text)))
    #list_words = " ".join(list(map(lambda x: try_str(x[19]), text)))
    list_words = list_words.lower().split(" ")
    # word_list = wordify(word_list).split(" ")
    list_words = list(w for w in list_words if len(w) > 1 and w not in stop_words)
    #list_words = get_list_of_words(text)
    word_dist = FreqDist(list_words)
    feature_set = word_dist.most_common(4000)
    word_dist = word_dist.most_common(10)
    dist = pd.DataFrame.from_records(word_dist).transpose
    print (messeg)
    print dist

    for word in feature_set:
        ret_list.append(word[0])

    return  ret_list



male_features = word_distrebution(males, 'Male most frequent words', lambda x: try_str(x[10])+try_str(x[19]))

print ()
female_features = word_distrebution(females, 'Female most frequent words', lambda x: try_str(x[10])+try_str(x[19]))

print ()
brand_features = word_distrebution(brands, 'Brand most frequent words', lambda x: try_str(x[10])+try_str(x[19]))


Male most frequent words
<bound method DataFrame.transpose of         0    1
0     i'm  360
1    like  317
2    love  297
3     get  257
4   don't  212
5     one  191
6    it's  190
7     new  169
8    time  169
9  people  159>
()
Female most frequent words
<bound method DataFrame.transpose of         0    1
0     i'm  463
1    like  379
2    love  377
3     get  254
4     one  230
5   don't  225
6     nan  222
7    life  198
8    it's  190
9  people  190>
()
Brand most frequent words
<bound method DataFrame.transpose of           0     1
0   weather  1095
1   updates   649
2  channel.   537
3    nanget   458
4      news   359
5        us   257
6       new   234
7    follow   188
8    latest   177
9       get   175>


Q2: we used :
    naive base classifier -  
                        without wordify 63.9%
                        with wordify    63.6%
    logistic regrassion   - 
                        without wordify 63.7%
                        with wordify    63.9%
                        
    keras model with embadding & lstm  -
          8 epoches , batch size =256  -66.76%
          4 epoches ,  batch size =256  -63.3%
          8 epoches, batch size =256  - 66.76%
          8 epoches, batch size = 500 - 63.83%
          8 epoches, batch size =100  - 64.96%
          
             
      
    
    wordify is a function that removes special characters
    
    

In [4]:
## Question 2
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
    return feature

def union(a, b, c):
    first = (set(a) | set(b))
    return list(first | set(c))

top_words = union(male_features, female_features, brand_features)

tweet_by_gender = (map(lambda x: (try_str(x[10])+ try_str(x[19]), x[5]),filtered))

feature_set = [(find_features(top_words, line[0]), line[1]) for line in tweet_by_gender]
training_set = feature_set[:int(len(feature_set)*4/5)]
testing_set = feature_set[int(len(feature_set)*4/5):]

# creating a naive bayes classifier
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy = nltk.classify.accuracy(NB_classifier, testing_set)*100
print("Naive Bayes Classifier accuracy =", accuracy)
NB_classifier.show_most_informative_features(20)

# creating a logistic regression classifier
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
accuracy = nltk.classify.accuracy(LogisticRegression_classifier, testing_set)*100
print("Logistic Regression classifier accuracy =", accuracy)

('Naive Bayes Classifier accuracy =', 63.9623324882289)
Most Informative Features
                 updates = True            brand : female =    271.8 : 1.0
                channel. = True            brand : female =    245.9 : 1.0
                  nanget = True            brand : female =    209.2 : 1.0
                   dates = True            brand : female =    152.2 : 1.0
                  update = True            brand : female =    110.4 : 1.0
                 weather = True            brand : female =     96.9 : 1.0
                 channel = True            brand : female =     81.4 : 1.0
              continuous = True            brand : female =     53.0 : 1.0
               subscribe = True            brand : female =     46.5 : 1.0
                    date = True            brand : female =     43.6 : 1.0
                 husband = True             male : brand  =     42.5 : 1.0
                 leading = True            brand : female =     35.2 : 1.0
                hu

Deep learning predictor
    embadding in the first layer
    lstm in the hidden. 
    

In [5]:
# creating a neural network classifier
neural_data_set = list(filter(lambda x: x[1] != "brand", tweet_by_gender))
x = list(neural_data_set[i][0] for i in range (0, len(neural_data_set)))
encoder = LabelEncoder()
y = encoder.fit_transform(list(neural_data_set[i][1] for i in range(0, len(neural_data_set))))


max_words = 4000
k_tokenizer = keras_token(num_words=max_words)
k_tokenizer.fit_on_texts(x)
x = k_tokenizer.texts_to_sequences(x)
x = sequence.pad_sequences(x)

# treat the labels as categories
y = keras.utils.to_categorical(y, 2)



X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 42)



embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_words, embed_dim,input_length = x.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=8, batch_size=256)

#4 epoches, batch 256 - acc - 63.3
#8 epoches,batch 256 - acc - 66.76
#8 epoches, batch 500 - acc - 63.83
#8 epoches, batch 100 - acc -64.96

# Final evaluation of the model
validation_size = 500
X_validate = X_test[-validation_size:]
Y_validate = y_test[-validation_size:]
X_test = X_test[:-validation_size]
y_test = y_test[:-validation_size]

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))



C:\Users\ariel\Anaconda2\lib\site-packages\keras\layers\recurrent.py:1993: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54L, 128)          512000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 767,194
Trainable params: 767,194
Non-trainable params: 0
_________________________________________________________________
None


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Train on 8016 samples, validate on 2004 samples
Epoch 1/8
8016/8016 [==============================] - 43s 5ms/step - loss: 0.6766 - acc: 0.5618 - val_loss: 0.6568 - val_acc: 0.6003
Epoch 2/8
8016/8016 [==============================] - 38s 5ms/step - loss: 0.6270 - acc: 0.6704 - val_loss: 0.6325 - val_acc: 0.6437
Epoch 3/8
8016/8016 [==============================] - 33s 4ms/step - loss: 0.5283 - acc: 0.7471 - val_loss: 0.6524 - val_acc: 0.6592
Epoch 4/8
8016/8016 [==============================] - 36s 4ms/step - loss: 0.4251 - acc: 0.8059 - val_loss: 0.7157 - val_acc: 0.6302
Epoch 5/8
8016/8016 [==============================] - 33s 4ms/step - loss: 0.3629 - acc: 0.8453 - val_loss: 0.7457 - val_acc: 0.6617
Epoch 6/8
8016/8016 [==============================] - 33s 4ms/step - loss: 0.2996 - acc: 0.8774 - val_loss: 0.8374 - val_acc: 0.6552
Epoch 7/8
8016/8016 [==============================] - 34s 4ms/step - loss: 0.2508 - acc: 0.8983 - val_loss: 1.0191 - val_acc: 0.6492
Epoch 8/8
8016

Q3: Get live tweets from twitter 
The most popular terms in that data set, is nails, women, feminism, love etc..
We get this terms because we filter tweets that contain this words.

In [ ]:
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import json
import unicodecsv as csv
from nltk.probability import FreqDist
import pandas as pd
from nltk.corpus import stopwords
import string


# Collect live data


def getLiveData():
    consumer_key = 'uW7b9X2txbXPgXXPjHNxk3yvZ'
    consumer_secret = 'nxQ38FkZkkkf3OSrL1pZwYbRBXRhTeKVetQtpecxtOsW0R7Erz'
    access_token = '33584667-xIa3A136SDuAk32JNqkZZYKqnwUFA2s30hKs8qUAp'
    access_secret = '59nOtlV0fEuS9qeN0DeroMqsWQ45Qmcn0Os5IqmnNWDvd'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    class MyListener(StreamListener):
        def on_data(self, data):
            data2 = json.loads(data)
            if not data2['entities']['urls']:
                description = data2['user']['description']
                if not isinstance(description, basestring):
                    description = ""
                text = description + ' ' + data2['text']
                print(text)
                try:
                    with open('realTime.csv', 'a') as f:

                        writer = csv.writer(f, dialect='excel', encoding='utf-8')
                        writer.writerow([text])
                        return True
                except BaseException as e:
                    print("Error on_data: %s" % str(e))
                return True

        def on_error(self, status):
            print(status)
            return True

    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(languages=['en'],
                          track=['nail', 'nails', 'feminism', '#nails', '#feminism', '#female', '#princess',
                                 '#feelings',
                                 '#aprincess', '#fashiondesigner', '#future', '#passion', '#dedication', '#year',
                                 '#milliondollarlisting', '#fetuses', '#china', '#death', '#born', '#policy', '#info',
                                 '#didyouknow', '#instagram', '#insta', '#instagood', '#instafact', '#instapic',
                                 '#like4like', '#likeforlike', '#tag', '#tagsforlikes', '#instatag', '#instame'])


def readData():
    with open('realTime.csv') as f:
        allData = []
        reader = csv.reader(f, encoding='utf-8')
        for row in reader:
            allData.append(row[0])

        return allData


stop_words = stopwords.words('english') + list(string.punctuation) + ['RT', 'rt', '&amp;']


def word_distrebution(text, messeg, f):
    ret_list = []
    list_words = " ".join(list(map(f, text)))
    list_words = list_words.lower().split(" ")
    list_words = list(w for w in list_words if len(w) > 1 and w not in stop_words)
    word_dist = FreqDist(list_words)
    feature_set = word_dist.most_common(4000)
    word_dist = word_dist.most_common(10)
    dist = pd.DataFrame.from_records(word_dist).transpose
    print (messeg)
    print dist

    for word in feature_set:
        ret_list.append(word[0])

    return ret_list


i = readData()

word_distrebution(i, "real time twits", lambda x: x)
print len(i)


Q4: we took hashtags that we think that women use more than men and let our model to predict
    our model predicted that out of the 4400 tweets that we collected 52.6% are women.
    Since we didnt tagged the collected tweets its hard to draw solid conclusions, bud the model predicted that there are           more women than men.
    

In [11]:
# q 4  - testint the model on data that was collected in real time


raw_tweets = pd.read_csv('realTime.csv', encoding='utf8')
list_tweets =  list(raw_tweets.values[i][0] for i in range (0, len(raw_tweets.values)))

#
to_predict = k_tokenizer.texts_to_sequences(list_tweets)
to_predict = sequence.pad_sequences(to_predict, maxlen=54L)


predictions = model.predict(to_predict, batch_size = 200)

#round predictions were 0 is female
                     #  1 is male
rounded = [round(x[0]) for x in predictions]
num_of_men = rounded.count(1.0)
num_of_women = len(rounded) - num_of_men
print ('women percentage in collected tweets:' ,float(num_of_women)/len(rounded)*100)

('women percentage in collected tweets:', 52.62569832402234)
